# Imports

In [1]:
import pickle
import pandas as pd
import numpy as np
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score

## Importando resultados

In [2]:
def import_best_result(file_name):
    with open(file_name, "rb") as f:
        return pickle.load(f)

def import_fuzzy_partitions_from_csv(file_name):
    return pd.read_csv(file_name, decimal=',')

    
fac_result = import_best_result("data/melhor_resultado_fac.pickle")
fou_result = import_best_result("data/melhor_resultado_fou.pickle")
kar_result = import_best_result("data/melhor_resultado_kar.pickle")
todas_result = import_best_result("data/melhor_resultado_todas.pickle")


fac_fuzzy = import_fuzzy_partitions_from_csv("data/fuzzy_partitions_fac.csv")
fou_fuzzy = import_fuzzy_partitions_from_csv("data/fuzzy_partitions_fou.csv")
kar_fuzzy = import_fuzzy_partitions_from_csv("data/fuzzy_partitions_kar.csv")
todas_fuzzy = import_fuzzy_partitions_from_csv("data/fuzzy_partitions_todas.csv")

## Melhores resultados

In [3]:
print("Melhor custo pro fac", fac_result["cost"])
print("Melhor custo pro fou", fou_result["cost"])
print("Melhor custo pro kar", kar_result["cost"])
print("Melhor custo com as três matrizes", todas_result["cost"])

Melhor custo pro fac 3383.2055017732046
Melhor custo pro fou 1800.4341935576729
Melhor custo pro kar 1517.5559412009745
Melhor custo com as três matrizes 6665.410166578429


## Modified partition coefficient

In [4]:
def calc_partition_coefficient(membership_degree):
    n = membership_degree.shape[0]
    membership_degree = np.power(membership_degree, 2)
    return membership_degree.sum(axis = 1).sum()/n

def calc_modified_partition_coefficient(membership_degree):
    c = membership_degree.shape[1]
    vpc = calc_partition_coefficient(membership_degree)
    return 1 - (c/(c-1))*(1-vpc)

In [5]:
print('>> FAC (VIEW1)', calc_modified_partition_coefficient(fac_result["membership_degree"]))
print('>> FOU (VIEW2)', calc_modified_partition_coefficient(fou_result["membership_degree"]))
print('>> KAR (VIEW3)', calc_modified_partition_coefficient(kar_result["membership_degree"]))
print('>> TODAS (VIEW0)', calc_modified_partition_coefficient(todas_result["membership_degree"]))

>> FAC (VIEW1) 0.012656027846346563
>> FOU (VIEW2) 0.0003644924738648303
>> KAR (VIEW3) 0.0009918034380510532
>> TODAS (VIEW0) 0.0023883326831980467


## Entropy

In [6]:
def calc_partition_entropy(membership_degree):
    n = membership_degree.shape[0]
    membership_degree = np.log10(membership_degree) * membership_degree
    return -membership_degree.sum(axis = 1).sum()/n


In [7]:
print('>> FAC (VIEW1)', calc_partition_entropy(fac_result["membership_degree"]))
print('>> FOU (VIEW2)', calc_partition_entropy(fou_result["membership_degree"]))
print('>> KAR (VIEW3)', calc_partition_entropy(kar_result["membership_degree"]))
print('>> TODAS (VIEW0)', calc_partition_entropy(todas_result["membership_degree"]))

>> FAC (VIEW1) 0.9773769238834532
>> FOU (VIEW2) 0.9993079325937968
>> KAR (VIEW3) 0.9981081541310196
>> TODAS (VIEW0) 0.9955900696072825


## Partições crisp

In [8]:
from itertools import chain

def get_hard_patitions(membership_degree):
    """
        membership_degree: numpy array of shape N x K
    """
    
    u = membership_degree
    members = []
    K = membership_degree.shape[1]
    
    # Obtendo o índice do grupo em que cada elemento possui maior valor de pertencimento
    element_index_membership = u.argsort(axis=1)[:, -1]
    
    index_class = {}
    for k in range(K):
        # Para cada grupo k, extrai quais dos elementos possui maior grau de pertencimento para ele
        memb = np.where(element_index_membership == k)[0]
        members.append(memb)
        index_class.update({m:k for m in memb})
    
    predicted_classes = [index_class[m] for m in sorted(index_class.keys())]
    
    return members, predicted_classes

def get_instances_class(qtd=10):
    return list(chain(*[[i]*200 for i in range(qtd)]))

classses = get_instances_class(10)

In [9]:
membership_degree = todas_result["membership_degree"]
hard_members, predicted_classes = get_hard_patitions(membership_degree)

for group_number, group in enumerate(hard_members):
    print(group_number, group, len(group))

print("Total de elementos somados:",  sum([len(g) for g in hard_members]))

0 [ 203  205  209  212  221  223  228  231  232  234  240  243  245  248
  250  251  259  265  270  271  277  285  286  294  295  297  298  303
  310  314  317  320  327  341  343  350  357  363  366  368  374  377
  381  382  383  396  401  405  407  559  609  648  652  670  692  725
  783  801  804  805  806  811  812  814  816  817  819  824  826  827
  830  832  835  836  837  838  839  848  850  851  856  858  859  860
  863  864  867  870  872  873  874  876  880  881  884  886  887  889
  891  893  895  896  899  903  904  920  924  926  928  929  930  931
  933  934  936  937  940  953  955  963  966  968  969  971  973  975
  977  979  990  994  997  998 1046 1048 1076 1078 1106 1200 1220 1222
 1274 1295 1322 1327 1345 1358 1359 1392 1397 1609 1639 1758 1919] 153
1 [] 0
2 [ 200  201  202  204  210  211  213  215  216  217  219  220  224  225
  226  229  230  233  236  237  238  241  242  244  252  253  255  256
  258  260  263  272  274  276  278  280  287  289  292  293  304 

## Adjusted Rand Index

In [10]:
from sklearn.metrics import adjusted_rand_score

fac_crisp, fac_classes = get_hard_patitions(fac_result["membership_degree"])
fou_crisp, fou_classes = get_hard_patitions(fou_result["membership_degree"])
kar_crisp, kar_classes = get_hard_patitions(fou_result["membership_degree"])
todas_crisp, todas_classes = get_hard_patitions(todas_result["membership_degree"])

print('>> FAC (VIEW1)', adjusted_rand_score(fac_classes, classses))
print('>> FOU (VIEW2)', adjusted_rand_score(fou_classes, classses))
print('>> KAR (VIEW3)', adjusted_rand_score(kar_classes, classses))
print('>> TODAS (VIEW0)', adjusted_rand_score(todas_classes, classses))

>> FAC (VIEW1) 0.28955002143740005
>> FOU (VIEW2) 0.0993059031935507
>> KAR (VIEW3) 0.0993059031935507
>> TODAS (VIEW0) 0.22917666469730535


## F-measure

In [11]:
print('>> FAC (VIEW1)', f1_score(fac_classes, classses, average="micro"))
print('>> FOU (VIEW2)', f1_score(fou_classes, classses, average="micro"))
print('>> KAR (VIEW3)', f1_score(kar_classes, classses, average="micro"))
print('>> TODAS (VIEW0)', f1_score(todas_classes, classses, average="micro"))

>> FAC (VIEW1) 0.152
>> FOU (VIEW2) 0.04900000000000001
>> KAR (VIEW3) 0.04900000000000001
>> TODAS (VIEW0) 0.119


## Erro de classificação

In [12]:
print('>> FAC (VIEW1)', 1 - accuracy_score(fac_classes, classses))
print('>> FOU (VIEW2)', 1 - accuracy_score(fou_classes, classses))
print('>> KAR (VIEW3)', 1 - accuracy_score(kar_classes, classses))
print('>> TODAS (VIEW0)', 1 - accuracy_score(todas_classes, classses))

>> FAC (VIEW1) 0.848
>> FOU (VIEW2) 0.951
>> KAR (VIEW3) 0.951
>> TODAS (VIEW0) 0.881
